In [1]:
import onnxruntime as ort

providers = ["CPUExecutionProvider"]
sess_options = ort.SessionOptions()
onnx_sess = ort.InferenceSession(
    "exp/id-ID-Althaf-piper.onnx",
    sess_options=sess_options,
    providers=providers,
)

In [2]:
from g2p_id import G2p
from piper_train.phonemize import phonemes_to_ids, ALPHABETS

phonemizer = G2p()

def phonemize(text):
    phonemes = phonemizer(text)
    phonemes = "# #".join(["#".join(phn) for phn in phonemes])
    phonemes = phonemes.split("#")
    phoneme_id_map = ALPHABETS["id"]
    phoneme_ids = phonemes_to_ids(
        phonemes, phoneme_id_map=phoneme_id_map
    )
    return phoneme_ids

In [3]:
import numpy as np

text = "mereka membuka tas penjual itu dan melihat isinya ."
input_ids = phonemize(text)
input_lengths = np.array([len(input_ids)], dtype=np.int64)
input_ids = np.array([input_ids], dtype=np.int64)
scales = np.array([0.667, 1.0, 0.8], dtype=np.float32)

In [4]:
outputs = onnx_sess.run(
    None,
    {
        "input": input_ids,
        "input_lengths": input_lengths,
        "scales": scales,
        "speaker_id": None,
    },
)

In [5]:
audio = outputs[0]

In [6]:
import IPython.display as ipd

ipd.Audio(audio[0, 0, 0, :], rate=44100)

In [7]:
audio.shape

(1, 1, 1, 147712)

In [8]:
duration_s = audio.shape[-1] / 44100
duration_s

3.3494784580498864

In [9]:
!ls -lh /root/piper/src/python/exp/id-ID-Althaf-piper.onnx

-rw-r--r-- 1 root root 61M Aug  3 08:15 /root/piper/src/python/exp/id-ID-Althaf-piper.onnx
